# Reproducing plot from Body Orientation post-processed data set

The goal is to visualize the results from the Body Orientation assesment experiment. For doing so, the function to create the plot and the dataset are provided. 

First, import the libraries to create the plot. The plot contains location points with the body orientation represented by an arrow. The **Wedge** represents the field of view of the depth camera. 

In [5]:
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.colorbar as mcolorbar
import matplotlib.pyplot as plt
from matplotlib.patches import Wedge
%matplotlib widget

Additionally, to process the data set it is necessary to import the following libraries:

In [6]:
import seaborn as sns
import sklearn
import pandas as pd
import numpy as np

Next, we define the plotting configuration function. The function handles the titles and the colors per body orientation **type**.

In [7]:
def plot_conf(type):
    choices = {'back': 'dodgerblue', 'backd': 'navy', 'frontal': 'seagreen', 'frontald': 'lightseagreen'}
    titles = {'sider': 'Side Right',
               'backdr': 'Back Diagonal Right',
               'back': 'Back',
               'backdl': 'Back Diagonal Left',
               'sidel': 'Side Left',
               'frontaldl': 'Frontal Diagonal Left',
               'frontal': 'Frontal',
               'frontaldr': 'Frontal Diagonal Right',
               'pairinter':'Pair view intersection',
               'formations': 'Group movement stops'
               }
    ranges = {'sider': '[-22.5°,0°),[0°,22.5°)',
               'backdr': '[22.5°,67.5°)',
               'back': '[67.5°,112.5°)',
               'backdl': '[112.5°,157.5°)',
               'sidel': '[157.5°,180°),(-180°,157.5°]',
               'frontaldl': '[-157.5°, -112.5°)',
               'frontal': '[-112.5°, -67.5°)',
               'frontaldr': '[-67.5°, -22.5°)'
               }
        
    shade_color = choices.get(type, 'purple')
    title = titles.get(type, 'Body')
    range_title=ranges.get(type,' ')
    
    return shade_color, title, range_title

For drawing the calculated body orientation from the skeleton joints, we calculate the vectors for the quiver function. Later, we setup the depth sensor's field of view.

In [8]:
def angle_cloud(data_kinect, shade_color, title,range_title, type):
  
    U = np.cos((data_kinect['re_body_angle']) )
    V = np.sin((data_kinect['re_body_angle']) )

    sns.set()

    norm = matplotlib.colors.Normalize()
    norm.autoscale(data_kinect['re_body_angle'])
    cm = matplotlib.cm.viridis

    sm = matplotlib.cm.ScalarMappable(cmap=cm, norm=norm)
    sm.set_array([])

    fig, ax = plt.subplots()

    ax.quiver(data_kinect['shl_x'], data_kinect['shl_y'], U, V, angles=data_kinect['re_body_angle'], color=cm(norm(data_kinect['re_body_angle'])), units='xy',pivot='middle')

    ax.axis(xmin=-3, xmax=3)
    ax.axis(ymin=-0.5, ymax=4.5)

    fov = Wedge(center=(0, 0), r=4.895, theta1=55, theta2=125, color=shade_color, alpha=0.05)
    ax.add_artist(fov)

    plt.ylabel('(Y) Distance from Origin')
    plt.xlabel('(X) Distance from Origin')
    plt.title('Body Orientation' +type+': '+title+'\n Accepted angle range '+range_title)

    cax, _ = mcolorbar.make_axes(plt.gca())
    cb = mcolorbar.ColorbarBase(cax, cmap=matplotlib.cm.viridis, norm=norm)
    cb.set_label('Body Orientation angle')

    return plt

The data is available in the folder *data*. It's a postprocessed file containing information about the body detected:
- ID_exp: identifier experiment
- date_exp: date and time of the sample
- Description: evaluated body orientation
- ID_subject: unique body identifier
- height: body height
- time: hh:mm:ss.ms formated time for the sample
- joints: object with coordinates x,y,z for each body joint.
- posture: detected body posture
- re_body_angle: calculated body angle
- origin_x and y: transformed location coordinates in the global reference system
- Coordinates for shoulder joints (sh_x,sh_y) where *r* refers to right and *l* to left. 
- Coordinates for spine (sp_x, y)

In [9]:
data_kinect = pd.read_csv("data/frontaldl_processed_data_frontaldl.csv",sep=';') 
data_kinect.reset_index(drop=True)
print(data_kinect)


      ID_row     ID_exp                    date_exp description  ID_subject  \
0          0  AW_221120  2020-11-22 14:56:17.129877   frontaldl  4037934527   
1          3  AW_221120  2020-11-22 14:56:17.791148   frontaldl  4037934527   
2          5  AW_221120  2020-11-22 14:56:18.224701   frontaldl  4037934527   
3          8  AW_221120  2020-11-22 14:56:18.889080   frontaldl  4037934527   
4         13  AW_221120  2020-11-22 14:56:19.986436   frontaldl  4037934527   
...      ...        ...                         ...         ...         ...   
1617    1980  AW_171220  2020-12-17 05:12:45.384457   frontaldl  4037937090   
1618    1981  AW_171220  2020-12-17 05:12:45.603111   frontaldl  4037937090   
1619    1982  AW_171220  2020-12-17 05:12:45.835644   frontaldl  4037937090   
1620    1983  AW_171220  2020-12-17 05:12:46.069020   frontaldl  4037937090   
1621    1984  AW_171220  2020-12-17 05:12:46.278443   frontaldl  4037937090   

      height              time  \
0      1.771  14:

To create the plot, specify the type of Body Orientation that would be processed. In this case, we have the *Frontal Diagonal Left* dataset. With this, the colors and titles are defined:

In [10]:
type='frontaldl'
shade_color, title, range_title= plot_conf(type)
angle_cloud(data_kinect, shade_color, title, range_title, type).show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …